### Hybrid Search Langchain

In [16]:
!pip install --upgrade --quiet  pinecone-client pinecone-text pinecone-notebooks

In [17]:
api_key="pcsk_ipKy5_TPw8QjbUR6YfW1J5wX6C1fdBSaRBbZpHrA4S9DEzWPTqgr7ySvRmuAf7ykrHFPn"

In [18]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [19]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [20]:
index=pc.Index(index_name)
index

In [21]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [34]:
# Check nltk working or not 
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', download_dir='C:\\Users\\user\\nltk_data')
nltk.download('punkt_tab', download_dir='C:\\Users\\user\\nltk_data')
nltk.data.path.append('C:\\Users\\user\\nltk_data') 
print(word_tokenize("In 2023, I visited Paris."))

[nltk_data] Downloading package punkt to C:\Users\user\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
from pinecone_text.sparse import BM25Encoder

# Initialize the BM25 encoder
bm25_encoder = BM25Encoder().default()

# Sample sentences
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans",
]

# Fit the BM25 encoder to the sentences
bm25_encoder.fit(sentences)


100%|██████████| 3/3 [00:00<00:00, 46.41it/s]


In [39]:
# Save BM25 values to a JSON file
bm25_encoder.dump("bm25_values.json")
# Load the BM25 values into a new encoder instance
bm25_encoder_loaded = BM25Encoder().load("bm25_values.json")


In [40]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [41]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000024D3D3205B0>, index=<pinecone.data.index.Index object at 0x0000024D3D7F3F40>)

In [42]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]
)

100%|██████████| 1/1 [00:03<00:00,  3.64s/it]


In [43]:
retriever.invoke("What city did i visit first")

[]